## Attention is all you need 실습
- 본 코드는 기본적으로 Attention is all you need (NIPS 2017) 논문의 내용을 최대한 따릅니다.
- 다만 일부 코드의 경우 Attention is all you need 내용을 따르지 않고 최신 아키텍처를 반영한 부분도 존재합니다. (가령, Positional Embedding 부분)

#### <b>하이퍼 파라미터 정의</b>

In [198]:
model_config = {
    "max_len" : 128, # 시퀀스 최대 길이
    "batch_size" : 32 # 배치 크기
}

#### <b>데이터 전처리 (PreProcessing)</b>

**실행 커맨드**
```sh
conda install -n playground pytorch
conda install -n huggingface
```

- 허깅페이스 API를 이용해서 대표적인 영어-독어 데이터셋인 **Multi30k**를 불러옵니다.

In [144]:
from datasets import load_dataset

# Hugging Face에서 데이터셋 로드
dataset = load_dataset("bentrevett/multi30k")

In [157]:
train_dataset, validation_dataset, test_dataset = dataset['train'], dataset['validation'], dataset['test']

print(train_dataset[0])

{'en': 'Two young, White males are outside near many bushes.', 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}


- **Tokenizer** 및 **Vocab** 생성

In [149]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [151]:
# Word-level tokenizer 초기화
unknown_token = "<unk>"

# 영어 토크나이저 정의 (Word Level)
en_tokenizer = Tokenizer(WordLevel(unk_token=unknown_token))
en_tokenizer.pre_tokenizer = Whitespace()

# 독일어 토크나이저 정의 (Word Level)
de_tokenizer = Tokenizer(WordLevel(unk_token=unknown_token))
de_tokenizer.pre_tokenizer = Whitespace()

In [155]:
# 학습용 trainer 설정 (vocab 생성)
pad_token, sos_token, eos_token = "<pad>", "<sos>", "<eos>"
special_tokens = [unknown_token, pad_token, sos_token, eos_token]

trainer = WordLevelTrainer(special_tokens=special_tokens, min_frequency=2)

In [159]:
# tokenizer 학습 (문장 리스트 기반)
train_en = train_dataset['en']
train_de = train_dataset['de']

en_tokenizer.train_from_iterator(train_en, trainer=trainer)
de_tokenizer.train_from_iterator(train_de, trainer=trainer)

In [169]:
# vocab 확인

print("[EN] vocab size: {}".format(en_tokenizer.get_vocab_size()))
print("[DE] vocab size: {}".format(de_tokenizer.get_vocab_size()))

print("[EN] Sample EN vocab tokens: {}".format(list(en_tokenizer.get_vocab().keys())[:10]))
print("[DE] Sample DE vocab tokens: {}".format(list(de_tokenizer.get_vocab().keys())[:10]))

[EN] vocab size: 6203
[DE] vocab size: 8060
[EN] Sample EN vocab tokens: ['shave', 'exterior', 'serious', 'medium', 'four', 'onlookers', 'waterfront', 'meal', 'called', 'auditorium']
[DE] Sample DE vocab tokens: ['Schwimmweste', 'Jugendlicher', 'weicht', 'leisten', 'veranstaltet', 'Betonkonstruktion', 'Veranstaltung', 'Winterkleidung', 'verkehrsreiche', 'First']


In [181]:
# 특수 토큰 인덱스 체크

# <unk> : 0
# <pad> : 1
# <sos> : 2
# <eos> : 3
for special_token in special_tokens:
    print("[EN] special_token: {}, index: {}".format(special_token, en_tokenizer.get_vocab()[special_token]))
    print("[DE] special token: {}, index: {}".format(special_token, de_tokenizer.get_vocab()[special_token]))

[EN] special_token: <unk>, index: 0
[DE] special token: <unk>, index: 0
[EN] special_token: <pad>, index: 1
[DE] special token: <pad>, index: 1
[EN] special_token: <sos>, index: 2
[DE] special token: <sos>, index: 2
[EN] special_token: <eos>, index: 3
[DE] special token: <eos>, index: 3


- 데이터 전처리
  - 데이터 패딩 등..

In [211]:
en_sos_id, en_eos_id = en_tokenizer.token_to_id(sos_token), en_tokenizer.token_to_id(eos_token)
de_sos_id, de_eos_id = de_tokenizer.token_to_id(sos_token), de_tokenizer.token_to_id(eos_token)

In [ ]:
def preprocess(data: dict) -> dict:
    # 토큰 id로 변환
    